In [1]:
from mqt.bench import CompilerSettings, QiskitSettings, get_benchmark
from qiskit import *
import numpy as np

min_qubit = 3 # maximum number of qubit
max_qubit = 5 # minimum number of qubit
gaps = 1
min_qubit = min(max(2, min_qubit), max_qubit) # maximum number of qubit
max_qubit = max(2, max_qubit) # minimum number of qubit
#skip_qubis = max(1, skip_qubits)
gaps = 1
compiler_settings = CompilerSettings(qiskit=QiskitSettings(optimization_level=1))
circuits= []
for i in range(min_qubit, max_qubit, gaps):
    qc = get_benchmark(benchmark_name="portfolioqaoa",
                       level="nativegates",
                       circuit_size=i,
                       compiler="qiskit",
                       compiler_settings=compiler_settings,
                       provider_name="ionq",)
    circuits.append((i,qc))
    print("---------------vqe with qubit", i,"---------------")
print("=================================================================================================")
print(circuits)

---------------vqe with qubit 3 ---------------
---------------vqe with qubit 4 ---------------
[(3, <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x747e0e16ca70>), (4, <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x747e9d92d730>)]


In [2]:
from qiskit_aer import AerSimulator
from qiskit.quantum_info import Statevector

simulator = AerSimulator()

jobs = []
for qubit_number, circuit_object in circuits:
    # Run noisy simulation (counts)
    job = simulator.run(circuit_object, shots=1000)
    results = job.result()
    counts = results.get_counts()

    # Get ideal distribution from statevector
    qc_nom = circuit_object.remove_final_measurements(inplace=False)
    state = Statevector.from_instruction(qc_nom)
    ideal_probs = state.probabilities_dict()

    jobs.append((qubit_number, counts, ideal_probs))

print("Collected results:")
for res in jobs:
    print(res)


Collected results:
(3, {'101': 5, '110': 8, '011': 6, '111': 7, '100': 293, '000': 48, '010': 337, '001': 296}, {'000': 0.050611836321212765, '001': 0.3073248399833461, '010': 0.30916132891268916, '011': 0.008929894430543883, '100': 0.3047594060440712, '101': 0.0061373359089927845, '110': 0.007114182325192089, '111': 0.005961176073951349})
(4, {'1110': 7, '1111': 9, '1101': 13, '0010': 8, '0000': 14, '1011': 4, '0011': 152, '0111': 3, '0101': 174, '1001': 166, '1100': 149, '1010': 122, '1000': 9, '0110': 151, '0100': 7, '0001': 12}, {'0000': 0.012044522341875061, '0001': 0.011268640518491056, '0010': 0.007864501783857211, '0011': 0.14318372451629274, '0100': 0.011159636491578254, '0101': 0.165754146018092, '0110': 0.1433637080005151, '0111': 0.008075302714678354, '1000': 0.009901777527396618, '1001': 0.15787934492114894, '1010': 0.13675504954022594, '1011': 0.007144948349164059, '1100': 0.15753258696431705, '1101': 0.010414249936295998, '1110': 0.007207256152968772, '1111': 0.010450604

In [3]:
# Hellinger fidelity function
def hellinger_fidelity_with_expected(p, q):
    """Compute Hellinger fidelity between measured distribution p (counts)
       and expected distribution q (probabilities)."""
    p_sum = sum(p.values())
    p_norm = {k: v / p_sum for k, v in p.items()}

    q_sum = sum(q.values())
    q_norm = {k: v / q_sum for k, v in q.items()}

    overlap = 0.0
    for k in set(p_norm) | set(q_norm):
        overlap += np.sqrt(p_norm.get(k, 0) * q_norm.get(k, 0))
    return overlap**2

In [4]:
for qubit_number, counts, ideal_probs in jobs:
    fidelity = hellinger_fidelity_with_expected(counts, ideal_probs)
    print(f"Hellinger Fidelity (QAOA, {qubit_number} qubits): {fidelity:.4f}")

Hellinger Fidelity (QAOA, 3 qubits): 0.9987
Hellinger Fidelity (QAOA, 4 qubits): 0.9966
